<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timevae_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install synthcity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/2

# Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [2]:
# Define file path
file_path = "/content/drive/Shareddrives/sp_env/datasets/Weather/weather.csv"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print(df.isnull().sum())

                  date  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0  2020-01-01 00:10:00   1008.89      0.71    273.18        -1.33    86.1   
1  2020-01-01 00:20:00   1008.76      0.75    273.22        -1.44    85.2   
2  2020-01-01 00:30:00   1008.66      0.73    273.21        -1.48    85.1   
3  2020-01-01 00:40:00   1008.64      0.37    272.86        -1.64    86.3   
4  2020-01-01 00:50:00   1008.61      0.33    272.82        -1.50    87.4   

   VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  ...  wv (m/s)  \
0          6.43          5.54          0.89       3.42  ...      1.02   
1          6.45          5.49          0.95       3.39  ...      0.43   
2          6.44          5.48          0.96       3.39  ...      0.61   
3          6.27          5.41          0.86       3.35  ...      1.11   
4          6.26          5.47          0.79       3.38  ...      0.49   

   max. wv (m/s)  wd (deg)  rain (mm)  raining (s)  SWDR (W/m�)  \
0           1.60     224.3     

In [3]:
# Set seed for reproducibility
np.random.seed(42)

# Convert 'date' to datetime and set as index (preserves chronological order)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Exclude 'date' from feature selection (it's now the index anyway)
columns = df.columns  # 'date' is no longer in df.columns
selected_features = np.random.choice(columns, size=10, replace=False)

print(f"Selected features: {selected_features}")

# Keep only the selected features
df = df[selected_features]

Selected features: ['p (mbar)' 'PAR (�mol/m�/s)' 'raining (s)' 'T (degC)' 'sh (g/kg)'
 'VPmax (mbar)' 'wv (m/s)' 'Tdew (degC)' 'max. PAR (�mol/m�/s)'
 'SWDR (W/m�)']


In [4]:
print(df.head())

                     p (mbar)  PAR (�mol/m�/s)  raining (s)  T (degC)  \
date                                                                    
2020-01-01 00:10:00   1008.89              0.0          0.0      0.71   
2020-01-01 00:20:00   1008.76              0.0          0.0      0.75   
2020-01-01 00:30:00   1008.66              0.0          0.0      0.73   
2020-01-01 00:40:00   1008.64              0.0          0.0      0.37   
2020-01-01 00:50:00   1008.61              0.0          0.0      0.33   

                     sh (g/kg)  VPmax (mbar)  wv (m/s)  Tdew (degC)  \
date                                                                  
2020-01-01 00:10:00       3.42          6.43      1.02        -1.33   
2020-01-01 00:20:00       3.39          6.45      0.43        -1.44   
2020-01-01 00:30:00       3.39          6.44      0.61        -1.48   
2020-01-01 00:40:00       3.35          6.27      1.11        -1.64   
2020-01-01 00:50:00       3.38          6.26      0.49        

In [5]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training, 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)

# Sequence length for time-series data (dataset = hourly; 24 hours)
sequence_length = 36
temporal_data = []
observation_times = []

# Generate sequences from df_scaled_train only
for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window

dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

TimeSeriesDataLoader created with 3465 sequences


In [ ]:
print(len(df_scaled_train))  # Check the length of the dataframe
print(loader.dataframe())

3500
        seq_id  seq_time_id  seq_temporal_PAR (�mol/m�/s)  \
0            0            0                      0.000000   
1            0            1                      0.000000   
2            0            2                      0.000000   
3            0            3                      0.000000   
4            0            4                      0.000000   
...        ...          ...                           ...   
124735    3464           31                      0.379520   
124736    3464           32                      0.396154   
124737    3464           33                      0.397787   
124738    3464           34                      0.451815   
124739    3464           35                      0.408922   

        seq_temporal_SWDR (W/m�)  seq_temporal_T (degC)  \
0                       0.000000               0.114042   
1                       0.000000               0.127230   
2                       0.000000               0.148177   
3                       0.

In [ ]:
syn_model = Plugins().get("timevae")

[2025-05-01T08:18:03.503116+0000][1547][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-05-01T08:18:03.503116+0000][1547][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [ ]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeVAEPlugin
clipping_value: 0
compress_dataset: False
decoder_batch_norm: False
decoder_dropout: 0.01
decoder_n_layers_hidden: 2
decoder_n_units_hidden: 150
decoder_nonlin: leaky_relu
decoder_nonlin_out_continuous: tanh
decoder_nonlin_out_discrete: softmax
decoder_residual: True
device: cuda
embedding_penalty: 10
encoder: None
encoder_batch_norm: False
encoder_dropout: 0.1
encoder_max_clusters: 20
encoder_n_layers_hidden: 3
encoder_n_units_hidden: 300
encoder_nonlin: leaky_relu
expecting_conditional: False
fitted: False
gamma_penalty: 1
lr: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timevae
module_relative_path: ../time_series/plugin_timevae.py
moments_penalty: 100
n_iter: 1000
n_iter_print: 10
outcome_encoder: TabularEncoder(cat_encoder_params={'handle_unknown': 'ignore',
                                   'sparse_output': False},
               categorical_encoder='onehot',
               cont_encoder_params={'n_components': 20},
 

## fitting the model

In [ ]:
print(loader.shape)
# Train the model
syn_model.fit(loader)

(124740, 13)


In [ ]:
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/VAE_Weather.pkl', syn_model)

In [ ]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

(124740, 13)


In [ ]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeVAE/weather"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-n_3000.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeVAE/weather/050125-085757-timevaeplugin-n_3000.csv


# Evaluation

## Prerequisites

In [ ]:
selected_columns = ['seq_temporal_PAR (�mol/m�/s)', 'seq_temporal_SWDR (W/m�)', 'seq_temporal_T (degC)',	'seq_temporal_Tdew (degC)',	'seq_temporal_VPmax (mbar)',	'seq_temporal_max. PAR (�mol/m�/s)',	'seq_temporal_p (mbar)',	'seq_temporal_raining (s)',	'seq_temporal_sh (g/kg)',	'seq_temporal_wv (m/s)']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data.dataframe()[selected_columns].to_numpy()

In [ ]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

[[0.         0.         0.11404189 ... 0.         0.29210526 0.13278008]
 [0.         0.         0.12723041 ... 0.         0.29736842 0.05532503]
 [0.         0.         0.14817688 ... 0.         0.30263158 0.05117566]
 ...
 [0.39778664 0.36214804 0.57796742 ... 0.         0.65789474 0.38865837]
 [0.45181468 0.41948019 0.58960434 ... 0.         0.66578947 0.27109267]
 [0.40892219 0.37985731 0.60512025 ... 0.         0.67631579 0.31258645]] 
 ------------------------------------------------------- 
 [[0.55969194 0.16735992 0.63528333 ... 0.05576173 0.38321587 0.24738849]
 [0.0497281  0.18382783 0.19698515 ... 0.26311472 0.7505566  0.42824979]
 [0.180474   0.16735992 0.26041931 ... 0.51152689 0.49017772 0.38798876]
 ...
 [0.11980377 0.14982813 0.11016495 ... 0.03436267 0.03389352 0.04138451]
 [0.31936363 0.44767405 0.77762817 ... 0.05091336 0.5704801  0.5044388 ]
 [0.180474   0.44767405 0.4841849  ... 0.26311472 0.7505566  0.1886928 ]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(124

## Generate distance metrics

### Helper Functions

In [ ]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [ ]:
# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(124740, 10) (124740, 10)
0.23688909792139168
0.2719719388618763
0.0762417138134981
0.05476511070850991
0.09085458536974098
0.23815391921701956
0.04997520085512207
0.22812857509092654
0.05684131195725542
0.100859472522241
Wasserstein Distance Results:
{'seq_temporal_PAR (�mol/m�/s)': 0.23688909792139168, 'seq_temporal_SWDR (W/m�)': 0.2719719388618763, 'seq_temporal_T (degC)': 0.0762417138134981, 'seq_temporal_Tdew (degC)': 0.05476511070850991, 'seq_temporal_VPmax (mbar)': 0.09085458536974098, 'seq_temporal_max. PAR (�mol/m�/s)': 0.23815391921701956, 'seq_temporal_p (mbar)': 0.04997520085512207, 'seq_temporal_raining (s)': 0.22812857509092654, 'seq_temporal_sh (g/kg)': 0.05684131195725542, 'seq_temporal_wv (m/s)': 0.100859472522241}
5.504778274928425
5.737495614674456
10.290058391681391
11.859912164992515
10.899121662615698
5.124645057980104
12.473151583968384
3.2007951587861703
11.812102565594072
12.577014140504607
KL Divergence Results:
{'seq_temporal_PAR (�mol/m�/s)': 5.5047782749284

# LSTM downstream

In [6]:
filepath = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeVAE/weather/050125-085757-timevaeplugin-n_3000.csv"

In [9]:
print(f"real_data: {real_data.shape}, synthetic_data: {df_synth.shape}")

real_data: (3500, 10), synthetic_data: (124740, 10)


In [8]:
real_data = df_scaled_test
df_synth = pd.read_csv(filepath)

# Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")


In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [11]:
# Convert to tensors (float32 for PyTorch)
data_real = torch.tensor(real_data.values, dtype=torch.float32)
data_synth = torch.tensor(df_synth.values, dtype=torch.float32)

# ──────── Sequence builder ───────────
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)

In [12]:
# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


# ─── Model Init ─────────────────────────────────────────────
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=64)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")

Epoch 1: Train MSE = 0.061166
Epoch 2: Train MSE = 0.055942
Epoch 3: Train MSE = 0.062349
Epoch 4: Train MSE = 0.055855
Epoch 5: Train MSE = 0.056223
Epoch 6: Train MSE = 0.061403
Epoch 7: Train MSE = 0.055855
Epoch 8: Train MSE = 0.058057
Epoch 9: Train MSE = 0.054990
Epoch 10: Train MSE = 0.059917
Epoch 11: Train MSE = 0.058516
Epoch 12: Train MSE = 0.055999
Epoch 13: Train MSE = 0.060728
Epoch 14: Train MSE = 0.062357
Epoch 15: Train MSE = 0.054606
Epoch 16: Train MSE = 0.051315
Epoch 17: Train MSE = 0.057704
Epoch 18: Train MSE = 0.064060
Epoch 19: Train MSE = 0.057147
Epoch 20: Train MSE = 0.055788
Epoch 21: Train MSE = 0.060213
Epoch 22: Train MSE = 0.053340
Epoch 23: Train MSE = 0.055352
Epoch 24: Train MSE = 0.054120
Epoch 25: Train MSE = 0.053724
Epoch 26: Train MSE = 0.063190
Epoch 27: Train MSE = 0.056931
Epoch 28: Train MSE = 0.061038
Epoch 29: Train MSE = 0.053703
Epoch 30: Train MSE = 0.055803
Epoch 31: Train MSE = 0.052391
Epoch 32: Train MSE = 0.060201
Epoch 33: Train M

In [14]:
model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_mse = loss_fn(preds, y_test).item()
    test_mae = mean_absolute_error(y_test.numpy(), preds.numpy())

    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test MAE: {test_mae:.6f}")

Test MSE: 2.572263
Test MAE: 1.059710
